#### Libraries

In [16]:
import numpy as np

import pandas as pd
data=pd.read_csv('WebKB_version2.csv')
import nltk
stopwords = nltk.corpus.stopwords.words('english')
import string
import re

from nltk.stem import PorterStemmer
ps = PorterStemmer()


### Step 1. Global Mono From extracted dataset computed excel

In [17]:

PostGlobalMONO = np.genfromtxt('Modified_EMO_3.txt', delimiter=',', dtype='float16') #for 12418 features

PostGlobalMONO.shape

(7708,)

In [18]:
#Pre-processing
def clean_text(txt):
    txt = "".join([c for c in txt if c not in string.punctuation])
    tokens = re.split('\W+',txt)
    txt = [ps.stem(word) for word in tokens if word not in stopwords]
    return txt

In [19]:
# Only simulate min_df = 2 for extracting 12418  features

from sklearn.feature_extraction.text import CountVectorizer

cv_vect = CountVectorizer( analyzer = clean_text, lowercase=True)
#cv_vect = CountVectorizer(analyzer='word', encoding='utf-8', decode_error='strict', strip_accents=None, lowercase=True, preprocessor=None, tokenizer=None, stop_words='english', token_pattern='(?u)\b\w\w+\b', max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False )

XCV = cv_vect.fit(data['document'])
X_CV = cv_vect.transform(data['document'])
X_CV.shape


(8324, 7708)

In [20]:
# features = vect.get_feature_names()  ---> simulate for 29k features

features = cv_vect.get_feature_names() #for 12418 

In [21]:
# Return terms per document with nonzero entries in X.
inv = cv_vect.inverse_transform(X_CV)

In [22]:
inv[0]


array(['09', '2', 'cornel', 'cours', 'cs', 'cs414', 'cs415', 'current',
       'edu', 'html', 'http', 'info', 'kb', 'webkb'], dtype='<U26')

In [23]:
PostGlobalMONO = PostGlobalMONO.astype("float16")

In [24]:
# Creating a Data Frame for Global Mono from generated computing in excel containing 2 columns = features and 1 + GlobalMONO * 6.0
df_PostGlobalMONO = pd.DataFrame(data={'Features': features, 'Global_Mono': PostGlobalMONO})

In [25]:
# displaying created dataframe
df_PostGlobalMONO

,Features,Global_Mono
0,,1.000000
1,0,2.269531
2,0002,1.000000
3,001,1.000000
4,002,1.004883
...,...,...
7703,zou,1.000000
7704,zpl,1.000000
7705,zuelzk,1.000000
7706,zymurgi,1.000000


In [26]:
# tokenizing each row and transfering to row list
row = data['document'].apply(lambda x: clean_text(x)) 
#row = cv_vect.inverse_transform(X_CV)

In [27]:
row[0]

['webkb',
 'cours',
 'cornel',
 'http',
 'cs',
 'cornel',
 'edu',
 'info',
 'cours',
 'current',
 'cs415',
 'cs414',
 'html',
 '2',
 '09',
 'kb']

# Auto Loop for  AVE (Generating TF AVE(MO)NO 12418 Feature Vectors) in 7512 rows

In [28]:
temp = np.zeros((1,7708), dtype="float16")
TF_MONO_TABLE = pd.DataFrame(np.zeros((1,7708 )))

#limit = 8324
#6 minutes is 1k rows
for i in range(0,8323):
    for j in row[i]:
        
        tf = row[i].count(j) / len(row[i])
        #print(tf)
       
        #print("tf OF {} = {} / {} = {}".format(j,data['document'][i].count(j),len(row[i]),tf))
        
    
        
        index_of_term = np.where(df_PostGlobalMONO == j)
        actual_index = index_of_term[0]
        
        # MONOfromDataFrame Represent the MONO in every row / document:
        MONOfromDataFrame = df_PostGlobalMONO['Global_Mono'].iloc[actual_index]
      
    
        TF_MONO_result = tf * MONOfromDataFrame
        #print("[Row {}] TF OF {} = {} / {} = {}".format(i+1,j,row[i].count(j),len(row[i]),tf))
        #print("* Global MONO of ")
        #print(df_PostGlobalMONO['Features'].iloc[actual_index])
        #print(MONOfromDataFrame)
        #print("The result is {}".format(TF_MONO_result))
        #print("------------------------------------------------------------")
        #print(j,tf, df_PostGlobalMONO['Global_Mono'].iloc[actual_index], TF_MONO_result)
        
        temp[0][actual_index] = TF_MONO_result
        
        # Coverting Temp Array into Data Frame
#Modify here 6:
    First_Result = pd.DataFrame(temp, columns = features)
    First_Result = First_Result.astype("float16")
    temp = np.zeros((1,7708), dtype="float16")
    
    TF_MONO_TABLE = np.append(TF_MONO_TABLE, np.array(First_Result), axis=0)
    
    First_Result.drop(First_Result.index[:], inplace=True)
# appending another dataframe as another row

#TF_MONO_TABLE2 = pd.DataFrame(np.ones((1,29523)))

#Modify here 7:

#TF_MONO_TABLE = pd.DataFrame(Second_Result, columns = features)   
TF_MONO_TABLE.shape

(8324, 7708)

# Save the as numpy array

In [29]:
TF_MONO_TABLE = TF_MONO_TABLE.astype("float16")

np.save("Modified_EMO_3_FULL_VALUES.npy",TF_MONO_TABLE)

# Clear all values in array and dataframe

In [30]:

TF_MONO_TABLE = np.array([])

#clear the array
temp = np.zeros((1,7708), dtype="float16")
temp.shape

(1, 7708)